In [1]:
import pandas as pd
import numpy as np
import requests
import sys

In [2]:
df = pd.read_csv('../data/extracted_data.csv')
df[['chr', 'start', 'id', 'ref', 'alt']]

,chr,start,id,ref,alt
0,1,1052729,mut1831,T,A
1,1,1227399,mut2068,G,A
2,1,1234820,mut1462,C,G
3,1,1256438,mut1952,C,T
4,1,1342375,mut943,A,G
...,...,...,...,...,...
3100,X,153006175,mut742,T,G
3101,X,153174592,mut1820,C,T
3102,X,153207408,mut1772,T,G
3103,X,153698355,mut1225,C,T


In [ ]:
resdf = pd.DataFrame()
idx = 0
while idx < len(df):
    data_str = '{ "variants" : ['
    while True and idx < len(df):
        data_str = data_str + '"{}  {}  {}  {}  {}  .   .   .",'.format(df.iloc[idx]['chr'], df.iloc[idx]['start'], df.iloc[idx]['id'], df.iloc[idx]['ref'], df.iloc[idx]['alt'])
        idx = idx + 1
        if idx % 200 == 0:
            break
    data_str = data_str[:-1]
    data_str = data_str + ']}'
    server = "https://grch37.rest.ensembl.org"
    ext = "/vep/homo_sapiens/region?GO=True&SpliceAI=True&dbscSNV=True&protein=True&uniprot=True"
    headers={ "Content-Type" : "application/json", "Accept" : "application/json"}
    r = requests.post(server+ext, headers=headers, data=data_str) # '{ "variants" : ["21  26960070  rs116645811 G A . . .", "21  26965148  rs1135638 G A . . ." ] }'
    
    if not r.ok:
        r.raise_for_status()
        sys.exit()
    
    decoded = r.json()
    print(repr(decoded))
    resdf = pd.concat([resdf, pd.DataFrame(decoded)])

In [4]:
resdf#.to_pickle('vep_api_output.pkl')

,allele_string,most_severe_consequence,regulatory_feature_consequences,strand,seq_region_name,input,transcript_consequences,id,assembly_name,end,start,colocated_variants,motif_feature_consequences,intergenic_consequences,minimised
0,T/A,upstream_gene_variant,[{'consequence_terms': ['regulatory_region_var...,1,1,1 1052729 mut1831 T A . . .,"[{'transcript_id': 'ENST00000294576', 'uniparc...",mut1831,GRCh37,1052729,1052729,NaN,NaN,NaN,NaN
1,G/A,3_prime_UTR_variant,"[{'variant_allele': 'A', 'regulatory_feature_i...",1,1,1 1227399 mut2068 G A . . .,"[{'hgnc_id': 10601, 'impact': 'MODIFIER', 'tra...",mut2068,GRCh37,1227399,1227399,"[{'phenotype_or_disease': 1, 'strand': 1, 'all...","[{'motif_pos': 1, 'impact': 'MODIFIER', 'high_...",NaN,NaN
2,C/G,non_coding_transcript_exon_variant,"[{'impact': 'MODIFIER', 'variant_allele': 'G',...",1,1,1 1234820 mut1462 C G . . .,"[{'strand': -1, 'gene_symbol_source': 'HGNC', ...",mut1462,GRCh37,1234820,1234820,"[{'end': 1234820, 'id': 'COSV61222486', 'seq_r...",NaN,NaN,NaN
3,C/T,stop_gained,NaN,1,1,1 1256438 mut1952 C T . . .,"[{'transcript_id': 'ENST00000323275', 'hgnc_id...",mut1952,GRCh37,1256438,1256438,"[{'end': 1256438, 'id': 'COSV59672305', 'seq_r...",NaN,NaN,NaN
4,A/G,missense_variant,"[{'impact': 'MODIFIER', 'regulatory_feature_id...",1,1,1 1342375 mut943 A G . . .,"[{'amino_acids': 'C/R', 'codons': 'Tgc/Cgc', '...",mut943,GRCh37,1342375,1342375,"[{'var_synonyms': {'COSMIC': ['COSM4703202']},...",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,T/G,splice_donor_variant,"[{'regulatory_feature_id': 'ENSR00001770375', ...",1,X,X 153006175 mut742 T G . . .,"[{'biotype': 'protein_coding', 'gene_symbol_so...",mut742,GRCh37,153006175,153006175,"[{'seq_region_name': 'X', 'id': 'COSV99497660'...",NaN,NaN,NaN
101,C/T,missense_variant,"[{'regulatory_feature_id': 'ENSR00002105717', ...",1,X,X 153174592 mut1820 C T . . .,"[{'gene_symbol_source': 'HGNC', 'variant_allel...",mut1820,GRCh37,153174592,153174592,"[{'allele_string': 'COSMIC_MUTATION', 'end': 1...",NaN,NaN,NaN
102,T/G,missense_variant,"[{'regulatory_feature_id': 'ENSR00000424671', ...",1,X,X 153207408 mut1772 T G . . .,"[{'protein_start': 241, 'hgnc_id': 9959, 'cons...",mut1772,GRCh37,153207408,153207408,"[{'var_synonyms': {'COSMIC': ['COSM6364490']},...",NaN,NaN,NaN
103,C/T,missense_variant,NaN,1,X,X 153698355 mut1225 C T . . .,"[{'gene_symbol_source': 'HGNC', 'impact': 'MOD...",mut1225,GRCh37,153698355,153698355,"[{'phenotype_or_disease': 1, 'id': 'COSV105281...",NaN,NaN,NaN


1.5 min for 1 call (300 mutations)

In [8]:
# resdf = pd.read_csv('../data/VEP/vep_api_output.csv')
resdf

,input,seq_region_name,end,assembly_name,strand,most_severe_consequence,start,regulatory_feature_consequences,transcript_consequences,allele_string,id,colocated_variants,motif_feature_consequences,intergenic_consequences,minimised
0,1 1052729 mut1831 T A . . .,1,1052729,GRCh37,1,upstream_gene_variant,1052729,"[{'variant_allele': 'A', 'biotype': 'promoter'...","[{'gene_symbol_source': 'HGNC', 'strand': -1, ...",T/A,mut1831,NaN,NaN,NaN,NaN
1,1 1227399 mut2068 G A . . .,1,1227399,GRCh37,1,3_prime_UTR_variant,1227399,"[{'regulatory_feature_id': 'ENSR00000249817', ...","[{'protein_id': 'ENSP00000321594', 'uniparc': ...",G/A,mut2068,"[{'allele_string': 'COSMIC_MUTATION', 'id': 'C...","[{'strand': 1, 'motif_name': 'ENSPFM0042', 'va...",NaN,NaN
2,1 1234820 mut1462 C G . . .,1,1234820,GRCh37,1,non_coding_transcript_exon_variant,1234820,"[{'regulatory_feature_id': 'ENSR00000918547', ...","[{'protein_id': 'ENSP00000321139', 'go': [{'go...",C/G,mut1462,"[{'start': 1234820, 'end': 1234820, 'seq_regio...",NaN,NaN,NaN
3,1 1256438 mut1952 C T . . .,1,1256438,GRCh37,1,stop_gained,1256438,NaN,"[{'cdna_start': 77, 'transcript_id': 'ENST0000...",C/T,mut1952,"[{'id': 'COSV59672305', 'allele_string': 'COSM...",NaN,NaN,NaN
4,1 1342375 mut943 A G . . .,1,1342375,GRCh37,1,missense_variant,1342375,"[{'variant_allele': 'G', 'biotype': 'promoter'...","[{'gene_symbol_source': 'HGNC', 'sift_predicti...",A/G,mut943,"[{'allele_string': 'COSMIC_MUTATION', 'id': 'C...",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,X 153006175 mut742 T G . . .,X,153006175,GRCh37,1,splice_donor_variant,153006175,[{'consequence_terms': ['regulatory_region_var...,"[{'variant_allele': 'G', 'uniparc': ['UPI00000...",T/G,mut742,"[{'var_synonyms': {'COSMIC': ['COSM8997094']},...",NaN,NaN,NaN
101,X 153174592 mut1820 C T . . .,X,153174592,GRCh37,1,missense_variant,153174592,"[{'biotype': 'open_chromatin_region', 'consequ...","[{'distance': 1975, 'protein_id': 'ENSP0000033...",C/T,mut1820,"[{'start': 153174592, 'strand': 1, 'allele_str...",NaN,NaN,NaN
102,X 153207408 mut1772 T G . . .,X,153207408,GRCh37,1,missense_variant,153207408,"[{'impact': 'MODIFIER', 'regulatory_feature_id...","[{'polyphen_prediction': 'possibly_damaging', ...",T/G,mut1772,"[{'phenotype_or_disease': 1, 'id': 'COSV641696...",NaN,NaN,NaN
103,X 153698355 mut1225 C T . . .,X,153698355,GRCh37,1,missense_variant,153698355,NaN,"[{'hgnc_id': 9101, 'gene_symbol': 'PLXNA3', 'g...",C/T,mut1225,"[{'var_synonyms': {'COSMIC': ['COSM9741561']},...",NaN,NaN,NaN


In [45]:
resdf.iloc[1]['transcript_consequences']

[{'consequence_terms': ['3_prime_UTR_variant'],
  'biotype': 'protein_coding',
  'gene_id': 'ENSG00000162572',
  'gene_symbol_source': 'HGNC',
  'strand': 1,
  'cdna_end': 2674,
  'gene_symbol': 'SCNN1D',
  'go': [{'description': 'sodium_channel_activity', 'go_term': 'GO:0005272'},
   {'go_term': 'GO:0005515', 'description': 'protein_binding'},
   {'go_term': 'GO:0005886', 'description': 'plasma_membrane'},
   {'description': 'sodium_ion_transport', 'go_term': 'GO:0006814'},
   {'go_term': 'GO:0015280',
    'description': 'ligand-gated_sodium_channel_activity'},
   {'description': 'membrane', 'go_term': 'GO:0016020'},
   {'go_term': 'GO:0016021', 'description': 'integral_component_of_membrane'},
   {'go_term': 'GO:0034220', 'description': 'ion_transmembrane_transport'},
   {'description': 'response_to_stimulus', 'go_term': 'GO:0050896'},
   {'description': 'sensory_perception_of_taste', 'go_term': 'GO:0050909'},
   {'go_term': 'GO:0055085', 'description': 'transmembrane_transport'}],
 

In [9]:
# resdf#.columns
import json
pd.json_normalize(resdf.regulatory_feature_consequences.iloc[0])

,variant_allele,biotype,impact,consequence_terms,regulatory_feature_id
0,A,promoter,MODIFIER,[regulatory_region_variant],ENSR00000000136


In [14]:
# resdf[[VEP_COLUMNS]] = np.nan
# resdf
for index, row in resdf.iterrows():
    rfc = pd.json_normalize(row['regulatory_feature_consequences'])
    resdf.at[index, rfc.iloc[0]['biotype']] = 1
    break

             consequence_terms regulatory_feature_id   biotype    impact  \
0  [regulatory_region_variant]       ENSR00000000136  promoter  MODIFIER   

  variant_allele  
0              A  


In [5]:
VEP_COLUMNS = ['ada_score', 'rf_score',
'ENSP', 'UNIPARC', 'GO',
'LOEUF', 'SpliceAI_pred_DP_AG', 'SpliceAI_pred_DP_AL',
'SpliceAI_pred_DP_DG', 'SpliceAI_pred_DP_DL', 'SpliceAI_pred_DS_AG',
'SpliceAI_pred_DS_AL', 'SpliceAI_pred_DS_DG', 'SpliceAI_pred_DS_DL',
'3_prime_UTR_variant', '5_prime_UTR_variant',
'NMD_transcript_variant', 
'downstream_gene_variant',
'intergenic_variant', 'intron_variant',
'non_coding_transcript_exon_variant', 'non_coding_transcript_variant',
'regulatory_region_variant',
'splice_donor_variant', 'splice_polypyrimidine_tract_variant',
'upstream_gene_variant', 'MODIFIER', 'MotifFeature',
'RegulatoryFeature', 'Transcript', 'CTCF_binding_site',
'enhancer', 'nonsense_mediated_decay',
'processed_pseudogene', 'processed_transcript', 'promoter',
'protein_coding', 'retained_intron',
'unprocessed_pseudogene',
'splice_acceptor_variant', 'splice_donor_region_variant', 'splice_donor_5th_base_variant',
'missense_variant', 'synonymous_variant', 'stop_gained', 'stop_lost', 'splice_region_variant', 'inframe_insertion', 'frameshift_variant']

In [41]:
df1['transcript_consequences'][1]

[{'consequence_terms': ['3_prime_UTR_variant'],
  'cdna_start': 2674,
  'transcript_id': 'ENST00000325425',
  'gene_id': 'ENSG00000162572',
  'gene_symbol': 'SCNN1D',
  'impact': 'MODIFIER',
  'cdna_end': 2674,
  'biotype': 'protein_coding',
  'gene_symbol_source': 'HGNC',
  'strand': 1,
  'go': [{'go_term': 'GO:0005272', 'description': 'sodium_channel_activity'},
   {'go_term': 'GO:0005515', 'description': 'protein_binding'},
   {'go_term': 'GO:0005886', 'description': 'plasma_membrane'},
   {'description': 'sodium_ion_transport', 'go_term': 'GO:0006814'},
   {'description': 'ligand-gated_sodium_channel_activity',
    'go_term': 'GO:0015280'},
   {'go_term': 'GO:0016020', 'description': 'membrane'},
   {'go_term': 'GO:0016021', 'description': 'integral_component_of_membrane'},
   {'description': 'ion_transmembrane_transport', 'go_term': 'GO:0034220'},
   {'go_term': 'GO:0050896', 'description': 'response_to_stimulus'},
   {'go_term': 'GO:0050909', 'description': 'sensory_perception_of